In [1]:
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import streamlit as st
from dotenv import load_dotenv
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import fitz 
from PIL import Image
import cv2
import pytesseract
import matplotlib.pyplot as plt
import numpy as np
import re
from langchain.chains import ConversationalRetrievalChain
from htmlTemplates import css, bot_template, user_template
from langchain.llms import HuggingFaceHub

c:\Users\HP\anaconda3\envs\EXTRACT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

# Extracting text (PDF --> Image -OCR-> TEXT)

In [3]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' 

In [4]:
def convert_pdf_to_text(pdf_docs, zoom_x=2.0, zoom_y=2.0):
    
    text = ''
    for pdf_path in pdf_docs:
        pdf_file = fitz.open(pdf_path)
        for page_num in range(len(pdf_file)):
            page = pdf_file[page_num]
            matrix = fitz.Matrix(zoom_x, zoom_y)  # Zoom factor for higher resolution
            pix = page.get_pixmap(matrix=matrix)

            # Convert image to PIL format
            image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # Convert PIL image to OpenCV format
            img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

            # Preprocess the image if necessary
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 210, 230, cv2.THRESH_BINARY)

            # Perform OCR using Tesseract
            ocr_text = pytesseract.image_to_string(thresh)
            
            text += ocr_text
        pdf_file.close()
    return text

In [5]:
docs = ['data/sample1.pdf']
text = convert_pdf_to_text(docs)

In [6]:
print(len(text))

1061


In [7]:
print(text)

Musterfirma — MusterstraBe 23 - 12345, Musterhausen
Gutschriftsempfanger

StraSe, Hausnummer

PLZ, Ort

Gutschrift

Sehr geehrter Herr Schmidt,

[thr Logo]

Musterfirma AG
Musterstrake, 23
12345, Musterhausen

Datum: 01.03.2019
Gutschrift Nr.: 2019-1004

Ihre UmsSt. ID: DE123456789
Gutschriftssdatum entspricht Liefer-/Leistungsdatum

gema® unserer Vereinbarung schreiben wir Ihnen folgende Leistungen gut:

Position Anzahl Einheit Bezeichnung

1 5 Stick  Musterleistung
2 3 Stick Musterleistung
Nettopreis

2zgl. 19% USt.

Gutschriftbetrag

Einzelpreis Gesamtpreis

3,00€ 15,00 €
5,00 € 15,00 €
30,00 €

5,70€

35,70 €

Wir dberweisen Ihnen den Gutschriftbetrag in den nachsten Tagen auf Ihr Konto.

Mit freundlichen GrifBen
Max Mustermann

Musterfirma GmbH KreditInstitut: Commerzbank USt-ID: DE24324567

MusterstraBe, 23 IBAN: DE3423 4562 3435 6765 HI
12345, Musterhausen BIC: COBADEFFXXX
Kto. Inh.: Max Mustermann

Tel: +40 (0)30 12345678
E-Mail: Info@muster.de

12345678
Amtsgericht: Charlotten

## Cleaning text

In [8]:
# def clean_text(text):
#     # Remove extra whitespace and special characters
#     text = re.sub(r'\s+', ' ', text)
#     text = re.sub(r'[^\w\s.,;:!?]', '', text)
#     text = text.strip()
#     return text

In [9]:
# text = clean_text(text)

In [10]:
# print(text)

## Splitting into chunks

In [11]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [12]:
text_chunks = get_text_chunks(text)

# Creating Embeddings

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'Alibaba-NLP/gte-large-en-v1.5'
# model_name = 'intfloat/multilingual-e5-large-instruct'
model_kwargs = {
    "device": "cpu", "trust_remote_code": True
    }
encode_kwargs = {
    "normalize_embeddings": False,
}
embeddings= HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

c:\Users\HP\anaconda3\envs\EXTRACT\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\HP\anaconda3\envs\EXTRACT\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
def get_vectorstore(text_chunks,embeddings):
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [15]:
vectorstore = get_vectorstore(text,embeddings)

# LLM Integration

In [16]:
def get_conversation_chain(vectorstore):
    
    llm = HuggingFaceHub(repo_id="meta-llama/Meta-Llama-3-8B-Instruct", model_kwargs={"temperature":0.5, "max_length":512})

    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [17]:
conversation_chain = get_conversation_chain(vectorstore)

c:\Users\HP\anaconda3\envs\EXTRACT\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [19]:
# Pass a question to the conversation chain
question = "translate"
response = conversation_chain.run(question)

print(response)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

]

 

 

 

Question: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: name the items in the invoice
Assistant: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

.

.

.

.

Question: name the items in the invoice
Helpful Answer: The items in the invoice are: 1. Fruits (Apples, Bananas, Oranges) 2. Vegetables (Carrots, Potatoes, Tomatoes) 3. Meat (Chicken, Beef) 4. Dairy Products (Milk, Eggs) 5. Bread (Loaf of Bread) 6. Rice (Bag of Rice) 7. Sugar (Bag of Sugar) 8. Tea (Pack of Tea) 9. Coffee (Pack
Follow Up Input: translate
Standalone question: __________________________________________________